In [24]:
import pandas as pd
import glob
import json
import numpy as np
from  tqdm import tqdm, trange

In [44]:
def label_BMEO(text,label_index):
    """

    Args:
        text: 列表形式的文字  例如：['腺', '样', '体', '肥', '大', '是', '临', '床']
        label_index: 标签对应的下标位置 [[1,3],[2,5]]  # 最后一个不要
    Returns:

    """
    label  = np.zeros(len(text),dtype=np.int8).tolist()
    for index in range(len(text)):
        for label_index_period in label_index:
            if index in range(label_index_period[0], label_index_period[1]):
                label[index] = 1
    text = pd.Series(text)
    label = pd.Series(label)
    new_pd = pd.DataFrame([text, label]).T
    new_pd.columns = ['text','label']
    new_pd.reset_index(inplace=True,drop=True)
    labels = new_pd['label'].values
    label_BMEO = []
    for i in range(len(labels)):
        three_values  = labels[i-1:i+2].tolist()
        if len(three_values) == 0:
            three_values = [0,labels[i],labels[i+1]]
        if three_values == [0,1,1]:
            label_BMEO.append('s-B')
        elif three_values == [1,1,1]:
            label_BMEO.append('s-M')
        elif three_values == [1,1,0]:
            label_BMEO.append('s-E')
        else:
            label_BMEO.append('o')
    new_pd['label_BMEO'] = pd.Series(label_BMEO)
    new_pd = new_pd[['text','label_BMEO']]
    return new_pd['text'].values, new_pd['label_BMEO']

def CMedCausal_pd(data_ls_i):
    text = list(data_ls_i['text'])
    label_index = []
    for value in data_ls_i['relation_of_mention']:
        try:
            temp = value['tail']
            label_index.append([temp['start_idx'],temp['end_idx']])
        except:
            temp = value['tail']['tail']
            label_index.append([temp['start_idx'],temp['end_idx']])

    return label_BMEO(text,label_index)

def save_pandas_file(file_train,pandas_file_name='CMedCausal'):
    file_train.columns = ['text','label_BMEO']
    out_train = file_train[:int(len(file_train)*0.95)]
    out_dev = file_train[int(len(file_train)*0.95):int(len(file_train) * 0.99)]
    out_test = file_train[int(len(file_train)*0.99):]
    out_train.to_csv(f'dataset/data_train_test_dev/train_{pandas_file_name}_bmeo.csv',index=False)
    out_dev.to_csv(f'dataset/data_train_test_dev/dev_{pandas_file_name}_bmeo.csv',index=False)
    out_test.to_csv(f'dataset/data_train_test_dev/test_{pandas_file_name}_bmeo.csv',index=False)
def index_to_text(text, index_ls):
    for value in index_ls:
        print(text[value[0]: value[1]])

In [26]:
data_ls = []
file_path = "dataset/医疗实体识别能用的数据/CMedCausal/CMedCausal_train.json"
for value in ['train','dev']:
    temp_path = file_path.split('_')
    temp_path_str = temp_path[0]+'_'+value+'.json'
    print(temp_path_str)
    with open(file_path, 'r', encoding='utf-8') as f:
        temp_data = json.load(f)
        data_ls.extend(temp_data)

dataset/医疗实体识别能用的数据/CMedCausal/CMedCausal_train.json
dataset/医疗实体识别能用的数据/CMedCausal/CMedCausal_dev.json


In [27]:
text_ls, label_index_ls = [],[]
for i in trange(len(data_ls)):
    text,label_index = CMedCausal_pd(data_ls[i])
    text_ls.append("".join(text.tolist()))
    label_index_ls.append("\t".join(label_index.tolist()))
CMedCausal_pd_file = pd.DataFrame([text_ls, label_index_ls]).T
# save_pandas_file(CMedCausal_pd_file,'CMedCausal')
CMedCausal_pd_file.columns = ['text','label_BMEO']

100%|██████████| 1600/1600 [00:10<00:00, 146.88it/s]


In [28]:
save_pandas_file(CMedCausal_pd_file,'CMedCausal')

。“临床表现”（tag为sym）
实体分为症状（tag为sym|sym-sd）和体征（tag为sym|sym-sn）三个子类.

In [29]:
data_ls = []
file_path = "dataset/医疗实体识别能用的数据/CMeEE/CMeEE_train.json"
for value in ['train','dev']:
    temp_path = file_path.split('_')
    temp_path_str = temp_path[0]+'_'+value+'.json'
    print(temp_path_str)
    with open(file_path, 'r', encoding='utf-8') as f:
        temp_data = json.load(f)
        data_ls.extend(temp_data)


dataset/医疗实体识别能用的数据/CMeEE/CMeEE_train.json
dataset/医疗实体识别能用的数据/CMeEE/CMeEE_dev.json


In [54]:
text_ls, label_index_ls = [],[]

for data_ls_i in tqdm(data_ls):
    text = data_ls_i['text']
    label_index = []
    for dict_value in data_ls_i['entities']:
        if dict_value['type'] == 'sym':
            label_index.append([dict_value['start_idx'], dict_value['end_idx']])
    label_index = [[value[0], value[1]+1] for value in label_index]
    # index_to_text(text, label_index)
    text = list(text)
    text, label_index = label_BMEO(text, label_index)
    text_ls.append("".join(text.tolist()))
    label_index_ls.append("\t".join(label_index.tolist()))

100%|██████████| 30000/30000 [01:05<00:00, 455.66it/s]


In [61]:
CMeEE_pd_file = pd.DataFrame([text_ls, label_index_ls]).T
CMeEE_pd_file

,0,1
0,（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
1,（6）发作一次伴血流动力学损害的室性心动过速（ventriculartachycardia）...,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
2,4.第三类（1）无症状性WPW综合征患者，年龄小于5岁。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
3,（2）室上性心动过速可用常规抗心律失常药物控制，年龄小于5岁。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
4,（3）非持续性，不考虑为无休止性的阵发性室性心动过速（即一次监视数小时或任何一小时记录的心电...,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
...,...,...
29995,Finder报道17例原发性支气管软化，均在6个月内发病，以喘鸣为主，均有左主支气管受累，两...,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
29996,本症亦可继发于其他原因，如气管插管时间过长、损伤、肿瘤和淋巴结压迫等。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
29997,本病以保守疗法为主，包括增强营养、适当补充维生素D及钙剂，保持呼吸道通畅。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
29998,支气管软化者应注意体位引流，可应用色甘酸、溴化异丙托品，但应避免使用β受体激动剂。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...


In [62]:
CMeEE_pd_file.columns = ['text','label_BMEO']
save_pandas_file(CMeEE_pd_file,'CMeEE')

In [63]:
file = pd.read_csv("dataset/data_train_test_dev/test_CMeEE_bmeo.csv")
texts, labels = file['text'], file['label_BMEO']

In [64]:
file

,text,label_BMEO
0,根据病史、症状、鼻腔检查及鼻窦华氏位摄影等检查，即可做出诊断。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
1,必要时进行CT检查。,o\to\to\to\to\to\to\to\to\to
2,对慢性顽固性鼻窦炎患儿可采用内镜鼻窦手术治疗。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
3,当人、畜接触疫水时，尾蚴很快（短至2～10秒钟）从皮肤或黏膜钻入体内，脱落尾部变为童虫。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
4,潜伏期内40%～75%的患者于接触疫水部位出现尾蚴性皮炎。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
...,...,...
295,Finder报道17例原发性支气管软化，均在6个月内发病，以喘鸣为主，均有左主支气管受累，两...,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
296,本症亦可继发于其他原因，如气管插管时间过长、损伤、肿瘤和淋巴结压迫等。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
297,本病以保守疗法为主，包括增强营养、适当补充维生素D及钙剂，保持呼吸道通畅。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...
298,支气管软化者应注意体位引流，可应用色甘酸、溴化异丙托品，但应避免使用β受体激动剂。,o\to\to\to\to\to\to\to\to\to\to\to\to\to\to\to...


In [65]:
len(list(texts[0])), len(labels[0].split('\t'))

(31, 31)